# 0. Imports

In [1]:
import pandas as pd
import numpy as np
import json
import multiprocessing

# 1. Get & Create Data

In [2]:
#Get & Format Data from content.csv
content = pd.read_csv('../raw_data/content.csv')
content_df = content.groupby('track_id', as_index=False).first()
content_df = content_df.reset_index(drop=True)
# content_df['track_name'] = content_df['track_name'].astype(str)
# content_df['artists'] = content_df['artists'].astype(str)
content_df = content_df[['track_id', 'track_name', 'artists']]

In [3]:
#Print Colab-Slices Locations (to check if they're correct)

slice_amount = 5
for i in range(slice_amount):
    #json_file_paths.append(f'../raw_data/playlist_data/mpd.slice.{i*1000}-{999+ i * 1000}.json')
    print((f'/mnt/d/spotify_million_playlist_dataset/data/mpd.slice.{i*1000}-{999+ i * 1000}.json'))

/mnt/d/spotify_million_playlist_dataset/data/mpd.slice.0-999.json
/mnt/d/spotify_million_playlist_dataset/data/mpd.slice.1000-1999.json
/mnt/d/spotify_million_playlist_dataset/data/mpd.slice.2000-2999.json
/mnt/d/spotify_million_playlist_dataset/data/mpd.slice.3000-3999.json
/mnt/d/spotify_million_playlist_dataset/data/mpd.slice.4000-4999.json
/mnt/d/spotify_million_playlist_dataset/data/mpd.slice.5000-5999.json
/mnt/d/spotify_million_playlist_dataset/data/mpd.slice.6000-6999.json
/mnt/d/spotify_million_playlist_dataset/data/mpd.slice.7000-7999.json
/mnt/d/spotify_million_playlist_dataset/data/mpd.slice.8000-8999.json
/mnt/d/spotify_million_playlist_dataset/data/mpd.slice.9000-9999.json
/mnt/d/spotify_million_playlist_dataset/data/mpd.slice.10000-10999.json
/mnt/d/spotify_million_playlist_dataset/data/mpd.slice.11000-11999.json
/mnt/d/spotify_million_playlist_dataset/data/mpd.slice.12000-12999.json
/mnt/d/spotify_million_playlist_dataset/data/mpd.slice.13000-13999.json
/mnt/d/spotify_m

In [4]:
# Specify the path to your JSON file
small_data_loc = "../raw_data/playlist_data/"
all_data_loc = "/mnt/d/spotify_million_playlist_dataset/data"

json_file_paths = [f'/mnt/d/spotify_million_playlist_dataset/data/mpd.slice.{i*1000}-{999+ i * 1000}.json'
                    for i in range(slice_amount)]

# Read the JSON file
json_data = []
for i in range(len(json_file_paths)):
    with open(json_file_paths[i], 'r') as file:
        json_data.append(json.load(file))

In [4]:
# # Specify the path to your JSON file
# json_file_paths = [f'/mnt/d/spotify_million_playlist_dataset/data/mpd.slice.{i*1000}-{999+ i * 1000}.json'
#                     for i in range(slice_amount)]

# def load_json_file(file_path):
#     with open(file_path, 'r') as file:
#         return json.load(file)

# pool = multiprocessing.Pool()
# json_data = pool.map(load_json_file, json_file_paths)
# pool.close()
# pool.join()

: 

: 

In [6]:
# #Get Track Sum per Batch
# track_sum_per_batch = 0
# batch = 0
# for i in range(len(json_data[batch]['playlists'])):
#     playlist_i = json_data[batch]['playlists'][i]['tracks']
#     track_sum_per_batch += len(playlist_i)

In [7]:
# #Get Track Sum
# track_sum = 0
# for i in range(len(json_data)):
#     json_i = json_data[i]['playlists']
#     for j in range(len(json_i)):
#         playlist = json_data[i]['playlists'][j]['tracks']
#         track_sum += len(playlist)

In [8]:
# #Get Track List Per Batch

# tracklist_per_batch = []
# batch = 0

# for i in range(len(json_data[batch]['playlists'])):
#     playlist_i = json_data[batch]['playlists'][i]['tracks']
#     for j in range(len(playlist_i)):
#         tracklist_per_batch.append(f'{i}' + ': ' + playlist_i[j]['artist_name'] + ' - ' + playlist_i[j]['track_name'])

In [5]:
# #Get Track List


# tracklist = []
# for k in range (len(json_data)):
#     for i in range(len(json_data[k]['playlists'])):
#         playlist_i = json_data[k]['playlists'][i]['tracks']
#         for j in range(len(playlist_i)):
#             tracklist.append(f'{k*1000 + i}' + ' ||| ' + playlist_i[j]['artist_name']
#                              + ' ||| ' + playlist_i[j]['track_name'] + ' ||| ' + str(playlist_i[j]['track_uri']))
            
# #Split Tracklist (to create DF)

# split_tracklist = [item.split(' ||| ') for item in tracklist]

# for i in range(len(split_tracklist)):
#     split_tracklist[i][3] = split_tracklist[i][3].replace('spotify:track:', '')

: 

: 

In [ ]:
tracklist = [f'{k*1000 + i} ||| {track["artist_name"]} ||| {track["track_name"]} ||| {track["track_uri"].replace("spotify:track:", "")}'
             for k, data in enumerate(json_data)
             for i, playlist in enumerate(data['playlists'])
             for track in playlist['tracks']]

split_tracklist = [item.split(' ||| ') for item in tracklist]
tracklist_df = pd.DataFrame(split_tracklist, columns=['playlist_number', 'artist_name', 'track_name', 'track_id'])

In [ ]:
# # Create DF
# tracklist_df = pd.DataFrame(split_tracklist, columns=['playlist_number', 'artist_name', 'track_name', 'track_id'])

In [ ]:
#Merge DFs on Track_ID (to get only the Tracks available in both Datasets)

merged_df = pd.merge(tracklist_df, content_df, on='track_id', how='inner')
clean_df = merged_df.drop(columns=['track_name_y', 'artists'])
clean_unique = clean_df.groupby('track_id', as_index=False).first()
merged_unique = clean_unique.reset_index(drop=True)

In [13]:
# #Create Dict per Track (to get all Playlists containing the Track)
# #Not operational

# trackdict = {}
# for i in range(10):
#     track = tracklist[i].split(' ||| ', tracklist[i])[1]
#     trackdict[f'{i}'] = []
#     for j in range(len(tracklist)):
#         if track in tracklist[j]:
#             trackdict[f'{i}'].append(tracklist[j].split(' ||| ')[0])
#     print(f'done with track number {i}')
# trackdict

# 2. Create Track_to_Playlist_Matrix


In [9]:
playlist_df = clean_df.groupby('track_id', as_index=False).agg({'playlist_number': list})
playlist_df = playlist_df.reset_index(drop=True)

In [10]:
# Get all playlists containing the track

# Specify the track_id you want to locate
target_track_id = '7zkLpY72g6lKQbiHDqri1S'

# Use loc[] to locate the rows with the specific track_id
specific_rows = playlist_df.loc[playlist_df['track_id'] == target_track_id]

playlistlist = specific_rows['playlist_number'].tolist()

In [11]:
# # One Hot Encode playlists into df

# # Get all unique playlists
# unique_playlists = []
# for i in range(len(playlist_df)):
#     unique_playlists += playlist_df['playlist_number'][i]
# unique_playlists = list(set(unique_playlists))
# unique_playlists.sort()

In [12]:
unique_playlists = playlist_df['playlist_number'].explode().unique()
unique_playlists.sort()

In [13]:
# Create track_to_playlist_matrix with shape:
# (len(playlist_df) = Amount of unique Tracks, len(unique_playlists) = Amount of unique Playlists)
# Index: (playlist_df['track_id'], unique_playlists)

track_to_playlist_matrix = np.zeros((playlist_df.shape[0], len(unique_playlists)))

for i, playlists in enumerate(playlist_df['playlist_number']):
    track_to_playlist_matrix[i, np.searchsorted(unique_playlists, playlists)] = 1

# 3 Create a Track_Matrix


In [14]:
import scipy.sparse as sp
%load_ext memory_profiler

## 3.1 Funktionen


In [ ]:
def old_matrix_function(track_to_playlist_matrix):
    # Create track_matrix with shape:
    # (len(playlist_df) = Amount of unique Tracks)**2
    # Index: (playlist_df['track_id'], playlist_df['track_id'])
    # with track_matrix[i][j] = amount of same playlists for track i and track j
    track_matrix = track_to_playlist_matrix @ track_to_playlist_matrix.T
    return track_matrix

In [ ]:
def new_matrix_function(track_to_playlist_matrix):
    # Convert track_to_playlist_matrix to sparse CSR matrix
    sparse_track_to_playlist_matrix = sp.csr_matrix(track_to_playlist_matrix)

    # Perform matrix multiplication using sparse matrix operations
    sparse_track_matrix = sparse_track_to_playlist_matrix @ sparse_track_to_playlist_matrix.T

    # Convert the sparse_track_matrix back to a dense matrix if needed
    dense_track_matrix = sparse_track_matrix.toarray()
    
    return dense_track_matrix

In [ ]:
def newnew_matrix_function(track_to_playlist_matrix, chunk_size=10):
    num_tracks = track_to_playlist_matrix.shape[0]
    num_chunks = int(np.ceil(num_tracks / chunk_size))

    # Initialize an empty sparse matrix to store the track matrix
    sparse_track_matrix = sp.csr_matrix((num_tracks, num_tracks))

    for chunk_idx in range(num_chunks):
        # Determine the range of track indices for the current chunk
        start_idx = chunk_idx * chunk_size
        end_idx = min((chunk_idx + 1) * chunk_size, num_tracks)

        # Extract the chunk of track-to-playlist matrix
        chunk_track_to_playlist_matrix = track_to_playlist_matrix[start_idx:end_idx]

        # Convert the chunk to a sparse CSR matrix
        sparse_chunk_track_to_playlist = sp.csr_matrix(chunk_track_to_playlist_matrix)

        # Perform matrix multiplication using sparse matrix operations
        sparse_chunk_track_matrix = sparse_chunk_track_to_playlist @ sparse_chunk_track_to_playlist.T

        # Accumulate the results in the overall track matrix
        sparse_track_matrix[start_idx:end_idx, start_idx:end_idx] = sparse_chunk_track_matrix
        
    # Convert the sparse track matrix back to a dense matrix if needed
    dense_track_matrix = sparse_track_matrix.toarray()

    return dense_track_matrix

## 3.2 Figuring out

## 3.2.1 Explore Datatypes and Multiplication types

In [ ]:
track_to_playlist_matrix.nbytes

272403840

In [ ]:
track_to_playlist_matrix_int16 = track_to_playlist_matrix.astype(np.int16)
track_to_playlist_matrix_int16.nbytes

68100960

In [ ]:
track_matrix_int16 = track_to_playlist_matrix_int16 @ track_to_playlist_matrix_int16.T

array([[1, 0, 0, ..., 0, 0, 0],
       [0, 3, 0, ..., 0, 0, 0],
       [0, 0, 2, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 2, 0],
       [0, 0, 0, ..., 0, 0, 2]], dtype=int16)

In [ ]:
# use np multiplier instead

track_matrix_int16_np = np.dot(track_to_playlist_matrix_int16,track_to_playlist_matrix_int16.T)

In [ ]:
track_matrix = track_to_playlist_matrix @ track_to_playlist_matrix.T

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 3., 0., ..., 0., 0., 0.],
       [0., 0., 2., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 2., 0.],
       [0., 0., 0., ..., 0., 0., 2.]])

In [ ]:
# use np multiplier instead

track_matrix_np = np.dot(track_to_playlist_matrix,track_to_playlist_matrix.T)

## 3.2.2 Use Dask to parallelize the matrix multiplication

In [15]:
import dask.array as da

In [16]:
# Load input matrices as Dask arrays
#track_to_playlist_da = da.from_array(track_to_playlist_matrix, chunks=chunk_size)
chunk_size_mb = 100  
chunk_size_bytes = chunk_size_mb * 2**20  
track_to_playlist_da = da.from_array(track_to_playlist_matrix, chunks=chunk_size_bytes)
x_transposed = track_to_playlist_da.transpose()
track_to_playlist_da_T = x_transposed.compute()

In [17]:
# Perform matrix multiplication
result = da.dot(track_to_playlist_da, track_to_playlist_da_T)

In [23]:
# Compute the result
result = result.compute()

In [18]:
def get_result(result):
    result = result.compute()
    return result

In [19]:
%memit get_result(result)

peak memory: 276.27 MiB, increment: 0.00 MiB


In [20]:
type(result)

dask.array.core.Array

## 3.2.3 Get Memory Usage

In [ ]:
%memit old_matrix_function(track_to_playlist_matrix_int16)

peak memory: 2110.63 MiB, increment: 0.00 MiB


In [ ]:
%memit new_matrix_function(track_to_playlist_matrix_int16)

peak memory: 2241.07 MiB, increment: 3.95 MiB


In [ ]:
%memit newnew_matrix_function(track_to_playlist_matrix_int16)

peak memory: 2235.46 MiB, increment: 0.00 MiB


## 3.3 Normalize n Stuff

In [ ]:
# give maximum value in matrix
max_value = np.max(track_matrix)
max_value

139.0

In [22]:
# Normalize the Matrix
def normalize(track_matrix):
    #normalize track_matrix
    for i in range(len(track_matrix)):
        if track_matrix[i, i] != 1:
            # Normalize values
            track_matrix[i, :] /= track_matrix[i, i]
            track_matrix[:, i] /= track_matrix[i, i]
    return track_matrix


In [23]:
result = normalize(result)

In [26]:
result

dask.array<sum-aggregate, shape=(0, 0), dtype=float64, chunksize=(0, 0), chunktype=numpy.ndarray>

# 4 Recommendations

In [ ]:
def create_colab_recommendations (track_matrix: np.ndarray, track_id):
    """
    Returns a list of recommended tracks for a given track_id
    Based on track_matrix
    (aka tracks that are in the same playlists as the given track_id)
    """
    idx = playlist_df.loc[playlist_df['track_id'] == target_track_id].index[0]
    track_matrix[idx]
    sorted_indices = np.argsort(-track_matrix[idx])
    sorted_values = track_matrix[idx][sorted_indices]
    merged_array = np.column_stack((sorted_indices, sorted_values))
    return merged_array

In [ ]:
def create_colab_recommendations(track_matrix: np.ndarray, target_track_id):
    """
    Returns a list of recommended tracks for a given track_id
    Based on track_matrix
    (aka tracks that are in the same playlists as the given track_id)
    """
    idx = np.where(playlist_df['track_id'] == target_track_id)[0][0]
    sorted_indices = np.argsort(-track_matrix[idx])
    merged_array = np.column_stack((sorted_indices, track_matrix[idx][sorted_indices]))
    return merged_array

hier könnten wir die daten normalisieren, also alle werte auf 0-1 bringen   
wenn ein track in vielen verschiedenen playlists landet   
(aktuelles maximum: 139x) könnte eine cooccurence weniger aussagekräftig sein   
dazu: jede cooccurence teilen   
durch: Wert auf der Diagonalen (also wie oft ein track in der playlist vorkommt)   
dann weighting Wert Alpha als Variable einbauen   